## Pipeline

1. Gather previous messages from raphraphraph, luckyjalien, and nil
2. Clean messages and put them into a dataframe
3. Process messages into embeddings 
    * Send messages (before embeddings) to cohere.ai to finetune already made models
    * Test Results
4. Use finetuned moddel to run with a discord bot command.
    * Input: A message
    * Output: The person most likely to type that message between raphraphraph, luckyjalien, and nil

## Libraries

In [1]:
import cohere
import csv
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

[Link to Guide that Helped Me](https://txt.cohere.ai/classify-three-options/)

## Cohere Embeddings and CSV for training on os.cohere.ai

In [2]:
from token_2 import TOKEN_COHERE
co = cohere.Client(TOKEN_COHERE)

In [3]:
# extension = 'csv'
# all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# combined_csv.to_csv( "messages.csv", index=False, encoding='utf-8')

In [4]:
df = pd.read_csv('messages.csv')

In [5]:
name_dict = {"luckyjalien#7103":"Jackie", "nil#7933":"Jun", "raphraphraph#1976":"Raph"}
df2 = df.replace({"author":name_dict})
df2 = df2.dropna()

In [6]:
X, y = df2["text"], df2["author"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
df2

,text,author
0,its fine,Jackie
1,oh lmfoa,Jackie
2,<:sweatduck:851002485157789736>,Jackie
3,it failed,Jackie
4,$history jackie,Jackie
...,...,...
1773,i wanna see him,Raph
1774,YES,Raph
1775,bet,Raph
1776,wha,Raph


In [8]:
def embed_text(text):
    output = co.embed(model='small', texts=text)
    return output.embeddings

In [9]:


X_train_emb = np.array(embed_text(X_train.tolist()))
X_test_emb = np.array(embed_text(X_test.tolist()))

In [10]:
#saving embeddings
np.savetxt('Xtrain.txt', X_train_emb)
np.savetxt('Xtest.txt', X_test_emb)

In [11]:
# Import modules
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score

# Prepare the labels
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_le = le.transform(y_train)
y_test_le = le.transform(y_test)

# Initialize the model
svm_classifier = SVC(class_weight='balanced')

# Fit the training dataset to the model
svm_classifier.fit(X_train_emb, y_train_le)

# Generate classification predictions on the test dataset
y_pred_le = svm_classifier.predict(X_test_emb)

# Compute metrics on the test dataset
accuracy = accuracy_score(y_test_le, y_pred_le)
f1 = f1_score(y_test_le, y_pred_le, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

Accuracy: 60.69
F1-score: 59.63


In [12]:
df3 = df2
df3 = df3.rename({"text":"Examples", "author":"Labels"}, axis = "columns")
df3

,Examples,Labels
0,its fine,Jackie
1,oh lmfoa,Jackie
2,<:sweatduck:851002485157789736>,Jackie
3,it failed,Jackie
4,$history jackie,Jackie
...,...,...
1773,i wanna see him,Raph
1774,YES,Raph
1775,bet,Raph
1776,wha,Raph


In [13]:
#df2.to_csv("cohere.csv", index_label = False,  index=False)

In [14]:
#send to training on Cohere
df3.to_csv("cohere1.csv", index_label = False,  index=False, header = False)

## Classify the Text on Testing data

In [15]:
#sent it to training on cohere
def classify_text_finetune(text):
    classifications = co.classify(
    model='ff3b9944-21b6-41de-bf58-dd5e4bd02993-ft',
    inputs=text
    )
    return classifications.classifications

In [16]:
# Generate classification predictions on the test dataset
y_pred_raw = classify_text_finetune(X_test.tolist())
y_pred = [y.prediction for y in y_pred_raw]

In [17]:
# Compute metrics on the test dataset
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

Accuracy: 85.52
F1-score: 85.54
